# Data Gathering and Cleaning

Data for this project is gathered from the U.S. Energy Information Administration (EIA) [website](https://www.eia.gov).

## Import packages

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import requests
import time
from bs4 import BeautifulSoup as BS

%load_ext autoreload
%autoreload 2
import helper_functions


We have to do a little manipulation to the text file from EIA. It is a text file with a bunch of line-separated JSON objects, but I massage it here to a proper JSON and export it as a new file.

In [24]:
lastline = None

# Open and read text file
with open("SEDS.txt","r") as f:
    lineList = f.readlines()
    
    # Keep track of last line
    lastline=lineList[-1]

# Open text file and create new json to be written
with open("SEDS.txt","r") as f, open("cleanfile.json","w") as g:
    
    # Iterate through each line of the text file
    for i,line in enumerate(f,0):
        
        # First line gets [ and , to initialize the json
        if i == 0:
            line = "["+str(line)+","
            g.write(line)
            
        # Last line gets ] to signal the end of the json
        elif line == lastline:            
            g.write(line)
            g.write("]")
            
        # Other lines get comma separation
        else:
            line = str(line)+","
            g.write(line)

In [33]:
file = open('cleanfile.json', 'r')
json_data = json.load(file)

The following energy types were selected based on the categories in the [EIA educational page](https://www.eia.gov/energyexplained/energy-and-the-environment/).

In [29]:
energy_types = [
    'All Petroleum Products excluding Fuel Ethanol',
    'Coal',
    'Natural Gas including Supplemental Gaseous Fuels',
    'Nuclear Power',
    'Biomass',
    'Fuel Ethanol excluding Denaturant',
    'Geothermal',
    'Hydroelectricity',
    'Solar Energy',
    'Wind Energy',
    'Renewable Energy'
]



# Make all lowercase in case some pages have inconsistent letter casing
for i in range(len(energy_types)):
    energy_types[i] = energy_types[i].lower()
    
nonrenewable_energies = energy_types[:4]

renewable_energies = energy_types[4:]

## Web scrape to get the information for each of the above types of energy

In [34]:
headers = {'user-agent': 'Safari/13.0.2 (Macintosh; Intel Mac OS X 10_15)'}
base_url = 'https://www.eia.gov/opendata/qb.php'
consumption_suffix = '?category=40204'

In [35]:
consumption_page = helper_functions.get_page(base_url+consumption_suffix,headers)

In [36]:
# Create empty dict to store all info across every sector and energy type by state
env_series_ids = {}

# Start by scraping the consumption website in order to get the list of available sectors  
consumption_sectors = consumption_page.find('div',{'class':'pagecontent mr_temp2'})

# Store sector url suffixes in a list
sector_url_suffixes = [sector.a['href'] for sector in consumption_sectors.find_all('li')[:7]]

# Loop 1 - iterate through each sector
for sector_url_suffix in sector_url_suffixes:
    
    # Scrape the sector page
    sector_page = helper_functions.get_page(base_url+sector_url_suffix,headers)    

    # Go into first url and grab tags of all children categories
    children_categories = sector_page.find('section').ul.find_all('li')

    # Store the urls of children cats (ccats = children categories)
    ccats_url_suffixes = [children_category.a['href'] 
                          for children_category in children_categories
                          if children_category.text.lower() in energy_types]
    
    # Loop 2 - for each sector, iterate through the relevant types of energy consumption to get state-level data
    for ccats_url_suffix in ccats_url_suffixes:
        
        # Scrape the child category page
        child_category_page = helper_functions.get_page(base_url+ccats_url_suffix,headers)

        # Grab tags of all energy unit children categories. Only want Btu
        energy_unit_cats = child_category_page.find('div',{'class':'main_col'}).ul.find_all('li')

        # Store only the url of the 'Btu' children category. I make a list and select only the first element 
        # because sometimes there will be two energy unit options or just one. This way ensures we only take 
        # the Btu option.
        btu_url_suffix = [energy_unit.a['href'] 
                   for energy_unit in energy_unit_cats
                   if energy_unit.text == 'Btu'][0]
        
        # Scrape the Btu page
        btu_page = helper_functions.get_page(base_url+btu_url_suffix,headers)
        
        # Get list of states by their tags
        states = btu_page.find('div',{'class':'main_col'}).ul.find_all('li')
        
        # Get url suffixes for each state
        state_url_suffixes = [state.a['href'] for state in states]
        
        # Isolate the sector and energy type
        sector = btu_page.find('div',{'class':'main_col'}).h3.find_all('a')[3].text
        energy_type = btu_page.find('div',{'class':'main_col'}).h3.find_all('a')[4].text
        
        # Add these to a dict which will be the values of the overarching env_series_ids dict
        series_id_values = {'sector':sector,'energy_type':energy_type}
        
        # Parse through url suffixes to get and store the series ids we want to use to parse the big JSON
        for state_suffix in state_url_suffixes:
            series_id = re.findall('SEDS.*',state_suffix)[0]
            env_series_ids[series_id] = series_id_values
            

### Parse energy data

In [37]:
# Set up empty bucket for parsed data
environmental_data = []

# Iterate through big json to parse relevant info
for single_json in json_data:
    
    # Only parse entries that have the series ids that we care about
    if single_json.get('series_id') in env_series_ids.keys():
        single_data_entry = {}
        single_data_entry['series_id'] = single_json['series_id']
        single_data_entry['sector'] = env_series_ids[single_json['series_id']]['sector']
        single_data_entry['data'] = single_json['data']
        single_data_entry['state'] = re.findall(', .*$',single_json['name'])[0][2:]
        single_data_entry['units'] = single_json['units']
        single_data_entry['energy_type'] = env_series_ids[single_json['series_id']]['energy_type']
        # Add a column for whether or not this type of energy is renewable
        if single_data_entry['energy_type'] in renewable_energies:
            single_data_entry['renewable'] = 1
        else: 
            single_data_entry['renewable'] = 0
        environmental_data.append(single_data_entry)

## Web scrape for population and GDP data

In [47]:
population_url_suffix = '?category=40367'
gdp_url_suffix = '?category=40828'
pop_gdp_url_suffixes = [population_url_suffix, gdp_url_suffix]

# Create container for  data
pop_gdp_series_ids = {}

for pop_gdp_url_suffix in pop_gdp_url_suffixes:
    
    # Scrape population page
    page = helper_functions.get_page(base_url + pop_gdp_url_suffix,headers)

    # Isolate html tags containing urls for each state
    state_tags = page.find('div',{'class':'main_col'}).ul.find_all('li')

    # Extract and save each state url suffix
    state_url_suffixes = [state_tag.a['href'] 
                          for state_tag in state_tags]

    # Iterate through each state url suffix to extract features
    for state_url_suffix in state_url_suffixes:

        # Scrape each state's series page
        state_page = helper_functions.get_page(base_url + state_url_suffix,headers)

        # Isolate html tags containing state name, get text from tag, parse for name
        api_call_tags = state_page.find('div',{'class':'main_col'}).find('div',{'class':'api_call_container'})
        state_text = api_call_tags.find_all('p')[1].text
        state = re.findall('(, )(.*)',state_text)[0][1]
        
        # Isolate html tags containing description (gdp or pop), get text from tag
        main_col_tags = state_page.find('div',{'class':'main_col'}).h3
        desc = main_col_tags.find_all('a')[2].text
        
        # Parse url suffix for series id
        series_id = re.findall('SEDS.*',state_url_suffix)[0]

        # Add to data container
        values = {'state':state,'description':desc}
        pop_gdp_series_ids[series_id] = values

### Parse population and gdp data

In [48]:
# Set up empty bucket for parsed data
pop_gdp_data = []

# Iterate through big json to parse relevant info
for single_json in json_data:
    
    # Only parse entries that have the series ids that we care about
    if single_json.get('series_id') in pop_gdp_series_ids.keys():
        single_data_entry = {}
        single_data_entry['series_id'] = single_json['series_id']
        single_data_entry['description'] = pop_gdp_series_ids[single_json['series_id']]['description']
        single_data_entry['units'] = single_json['units']
        single_data_entry['data'] = single_json['data']
        single_data_entry['state'] = pop_gdp_series_ids[single_json['series_id']]['state']
        pop_gdp_data.append(single_data_entry)

## Storing data into MongoDB

In [2]:
from pymongo import MongoClient

In [3]:
from pprint import pprint

In [6]:
client = MongoClient('mongodb://localhost/')

In [7]:
db = client.admin

In [8]:
# Issue the serverStatus command and print the results
serverStatusResult=db.command("serverStatus")
pprint(serverStatusResult)

{'asserts': {'msg': 0, 'regular': 0, 'rollovers': 0, 'user': 10, 'warning': 0},
 'connections': {'active': 1,
                 'available': 3273,
                 'current': 3,
                 'totalCreated': 6},
 'electionMetrics': {'averageCatchUpOps': 0.0,
                     'catchUpTakeover': {'called': 0, 'successful': 0},
                     'electionTimeout': {'called': 0, 'successful': 0},
                     'freezeTimeout': {'called': 0, 'successful': 0},
                     'numCatchUps': 0,
                     'numCatchUpsAlreadyCaughtUp': 0,
                     'numCatchUpsFailedWithError': 0,
                     'numCatchUpsFailedWithNewTerm': 0,
                     'numCatchUpsFailedWithReplSetAbortPrimaryCatchUpCmd': 0,
                     'numCatchUpsSkipped': 0,
                     'numCatchUpsSucceeded': 0,
                     'numCatchUpsTimedOut': 0,
                     'numStepDownsCausedByHigherTerm': 0,
                     'priorityTakeover': {'ca

In [4]:
import pymongo

In [9]:
mydb = client['energy_data']

In [396]:
print(client.list_database_names())

['admin', 'config', 'local']


In [10]:
energy_collection = mydb['energy_data']

In [45]:
env_results = energy_collection.insert_many(environmental_data)

In [49]:
pop_gdp_results = energy_collection.insert_many(pop_gdp_data)

### Practice querying

In [52]:
oregon = [x for x in energy_collection.find({'state':'Oregon'})]

In [134]:
oregon

[{'_id': ObjectId('5e17a25fedd91fc8283e8db8'),
  'series_id': 'SEDS.NGCCB.OR.A',
  'sector': 'Commercial Sector',
  'data': [['2018', 30969],
   ['2017', 33987],
   ['2016', 28570],
   ['2015', 26962],
   ['2014', 29185],
   ['2013', 30832],
   ['2012', 29451],
   ['2011', 31034],
   ['2010', 27461],
   ['2009', 30504],
   ['2008', 31202],
   ['2007', 29960],
   ['2006', 28840],
   ['2005', 28627],
   ['2004', 26438],
   ['2003', 26294],
   ['2002', 28417],
   ['2001', 28690],
   ['2000', 29481],
   ['1999', 30229],
   ['1998', 27285],
   ['1997', 26751],
   ['1996', 26712],
   ['1995', 23417],
   ['1994', 24024],
   ['1993', 25023],
   ['1992', 20318],
   ['1991', 23014],
   ['1990', 20913],
   ['1989', 20980],
   ['1988', 18829],
   ['1987', 17186],
   ['1986', 17214],
   ['1985', 19614],
   ['1984', 17624],
   ['1983', 15764],
   ['1982', 17049],
   ['1981', 15578],
   ['1980', 15872],
   ['1979', 14439],
   ['1978', 12671],
   ['1977', 11317],
   ['1976', 14499],
   ['1975', 16516]

In [137]:
dates = np.arange(2018,1959,-1)
df = pd.DataFrame(index = dates)

for series in oregon:
    if series.get('sector') == 'Residential Sector':
        print(series['energy_type'])
        ts_values = [tuple_[1] for tuple_ in series['data']]
        df = pd.concat([df, pd.Series(data = ts_values,name=series['energy_type'], index=dates)],axis=1)


Natural Gas including Supplemental Gaseous Fuels
Solar energy


ValueError: Length of passed values is 58, index implies 59

In [132]:
df.head()

,Natural Gas including Supplemental Gaseous Fuels,Solar energy,Geothermal,Coal
2018,30969,30969,30969,30969
2017,33987,33987,33987,33987
2016,28570,28570,28570,28570
2015,26962,26962,26962,26962
2014,29185,29185,29185,29185


In [82]:
series.keys()

dict_keys(['_id', 'series_id', 'sector', 'data', 'state', 'units', 'energy_type', 'renewable'])

In [117]:
test_df = pd.concat([or_df, pd.Series(values,name='test',index=dates)],axis=1)

In [118]:
pd.concat([test_df, pd.Series(values, name='test 2',index=dates)],axis=1)

,test,test 2
2018,30969,30969
2017,33987,33987
2016,28570,28570
2015,26962,26962
2014,29185,29185
2013,30832,30832
2012,29451,29451
2011,31034,31034
2010,27461,27461
2009,30504,30504


In [119]:
series['data']

[['2018', 30969],
 ['2017', 33987],
 ['2016', 28570],
 ['2015', 26962],
 ['2014', 29185],
 ['2013', 30832],
 ['2012', 29451],
 ['2011', 31034],
 ['2010', 27461],
 ['2009', 30504],
 ['2008', 31202],
 ['2007', 29960],
 ['2006', 28840],
 ['2005', 28627],
 ['2004', 26438],
 ['2003', 26294],
 ['2002', 28417],
 ['2001', 28690],
 ['2000', 29481],
 ['1999', 30229],
 ['1998', 27285],
 ['1997', 26751],
 ['1996', 26712],
 ['1995', 23417],
 ['1994', 24024],
 ['1993', 25023],
 ['1992', 20318],
 ['1991', 23014],
 ['1990', 20913],
 ['1989', 20980],
 ['1988', 18829],
 ['1987', 17186],
 ['1986', 17214],
 ['1985', 19614],
 ['1984', 17624],
 ['1983', 15764],
 ['1982', 17049],
 ['1981', 15578],
 ['1980', 15872],
 ['1979', 14439],
 ['1978', 12671],
 ['1977', 11317],
 ['1976', 14499],
 ['1975', 16516],
 ['1974', 13991],
 ['1973', 14541],
 ['1972', 15184],
 ['1971', 14173],
 ['1970', 11871],
 ['1969', 10503],
 ['1968', 8299],
 ['1967', 7372],
 ['1966', 7005],
 ['1965', 5983],
 ['1964', 5708],
 ['1963', 4505]